In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../project")

In [14]:
import wandb
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from common import testutils
from common import nxutils
from common.constants import *

In [25]:
task_in_size=2
task_out_size=1
task_description = (
    ('moons' , (task_in_size, task_out_size)), 
    ('circles', (task_in_size, task_out_size)), 
    ('spirals', (task_in_size, task_out_size))
)
num_tasks = len(task_description)
shape = [
    task_in_size*num_tasks,
    5*num_tasks,
    5*num_tasks,
    task_out_size*num_tasks
]

model = testutils.BaseModel(shape, init_bias_zero=True)
model = testutils.make_splitable_model(model, num_tasks, tile_offset= 0)
gm = nxutils.GraphManager(model, shape, task_description)

In [27]:
with wandb.init(project="test-plots") as run:
    run.log({"NetworkX Graph": gm.make_plotly()})

In [23]:
pos=nx.multipartite_layout(G, LAYER)

edge_x, edge_y = edges_from_pos(pos)
node_x, node_y = nodes_from_pos(pos)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(
        width=0.5, 
        #color=gm.get_edge_colors()
        ),
    hoverinfo='none',
    mode='lines'
)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        color=gm.get_node_colors(),
        line_width=2
        )
    )

# Create a Plotly figure
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        showlegend=False,
        hovermode='closest',
        margin=dict(b=0,l=0,r=0,t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    )

with wandb.init(project="test-plots") as run:
    run.log({"NetworkX Graph": fig})

In [9]:

# Create a NetworkX graph
G = nx.random_geometric_graph(100, 0.125)

# Get node positions from the graph
pos = nx.get_node_attributes(G, "pos")

# Create edges
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Create a trace for edges
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create nodes
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

# Create a trace for nodes
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=10,
        color=[],
        line_width=2))

# Create a Plotly figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0,l=0,r=0,t=0),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

with wandb.init(project="test-plots") as run:
    run.log({"NetworkX Graph": fig})


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mxmn. Use `wandb login --relogin` to force relogin


In [6]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(200, 0.125)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

### Logging as an Image.
This is ok, but I think plotly graphs are more performant and simply make more sense. 

In [ ]:

# Create a sample NetworkX graph
G = nx.Graph()
G.add_edges_from([(1, 2), (1, 3), (2, 4), (3, 4), (2, 3)])

# Plot the graph
plt.figure(figsize=(8, 6))
nx.draw(G, with_labels=True, node_color='skyblue', node_size=700, font_size=15, font_weight='bold')
plt.title("Sample NetworkX Graph")

plt.savefig("sample_graph.png")

with wandb.init(project="test-plots") as run:
    run.log({"NetworkX Graph": wandb.Image("sample_graph.png")})
